In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Demo to Show Anywidget mode

In [ ]:
import bigframes.pandas as bpd

Set the display option to use anywidget

In [ ]:
bpd.options.bigquery.ordering_mode = "partial"
bpd.options.display.repr_mode = "anywidget"

Load Sample Data

In [ ]:
df = bpd.read_gbq("bigquery-public-data.usa_names.usa_1910_2013")
print(df)

Display Series in anywidget mode

In [ ]:
test_series = df["year"]
print(test_series)

Display with Pagination

In [ ]:
df

### Sorting by Column\n
Click on any column header to sort the table by that column. Click again to toggle between ascending and descending order.

Programmatic Navigation Demo

In [ ]:
from bigframes.display.anywidget import TableWidget
import math
  
# Create widget programmatically  
widget = TableWidget(df)
print(f"Total pages: {math.ceil(widget.row_count / widget.page_size)}")
  
# Display the widget
widget

Test Navigation Programmatically

In [ ]:
# Simulate button clicks programmatically
print("Current page:", widget.page)

# Go to next page
widget.page = 1
print("After next:", widget.page)

# Go to previous page
widget.page = 0
print("After prev:", widget.page)

Edge Case Demonstration

In [ ]:
# Test with very small dataset
small_df = df.sort_values(["name", "year", "state"]).head(5)
small_widget = TableWidget(small_df)
print(f"Small dataset pages: {math.ceil(small_widget.row_count / small_widget.page_size)}")
small_widget

### Displaying Generative AI results containing JSON
The `AI.GENERATE` function in BigQuery returns results in a JSON column. While BigQuery's JSON type is not natively supported by the underlying Arrow `to_pandas_batches()` method used in anywidget mode ([Apache Arrow issue #45262](https://github.com/apache/arrow/issues/45262)), BigQuery Dataframes automatically converts JSON columns to strings for display. This allows you to view the results of generative AI functions seamlessly.

In [ ]:
bpd._read_gbq_colab("""
  SELECT
    AI.GENERATE(
      prompt=>(\"Extract the values.\", OBJ.GET_ACCESS_URL(OBJ.FETCH_METADATA(OBJ.MAKE_REF(gcs_path, \"us.conn\")), \"r\")),
      connection_id=>\"bigframes-dev.us.bigframes-default-connection\",
      output_schema=>\"publication_date string, class_international string, application_number string, filing_date string\") AS result,
    *
  FROM `bigquery-public-data.labeled_patents.extracted_data`
  LIMIT 5;
""")